<a class="anchor" id="section6"></a>

## Section 6 Evaluation

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
df_test = pd.read_csv('data/split/test_user_artists.csv')
user_based_result = pd.read_csv('data/result/user_based_result.csv')
item_based_result = pd.read_csv('data/result/item_based_result.csv')
personalrank_tags = pd.read_csv('data/result/withtag.csv')
personalrank_wo_tags = pd.read_csv('data/result/without_tag.csv')
content_based_result = pd.read_csv('data/result/content_based_result.csv')
df_train = pd.read_csv('data/split/train_user_artists.csv')

In [ ]:
merged_genre = pd.read_csv('data/external/merged_genre.csv')

In [ ]:
# Prepare the novelty for each artist
df_train = df_train[['name', 'userID',	'artistID',	'weight']]
artist_sum = df_train.groupby(by='artistID').count().drop(['name','weight'], axis=1).rename(columns={"userID": "user_count"})
artist_sum['novelty'] = artist_sum['user_count'].apply(lambda x: -1*math.log2(x/len(df_train.userID.unique()))/(-1*math.log2(1/len(df_train.userID.unique()))))
artist_sum.head()

,user_count,novelty
artistID,,
1,2,0.908136
2,7,0.742106
3,2,0.908136
4,2,0.908136
5,1,1.000000


In [ ]:
df_test = df_test[['userID','artistID', 'name',	'weight']]
print('{} unique user in test set'.format(len(df_test.userID.unique())))
df_test.head()

1876 unique user in test set


,userID,artistID,name,weight
0,175,356,Brad Paisley,2167
1,807,10242,詩月カオリ,75
2,904,1632,Metric,51
3,478,7339,AbraSKAdabra,179
4,2007,3780,D.R.I.,2294


In [ ]:
# Calculate Recall@10 and Precision@10 
def evaluation_metric(df_test, result):

    score = pd.DataFrame(columns={'userID','recall@10','precision@10'})
    fail = []
    user_list = result.userID.unique()
    for i in user_list:
        pred = result.loc[result['userID'] == i, 'artistID'].tolist()
        true = df_test.loc[df_test['userID'] == i].sort_values(by=['weight'],ascending = False)
        if len(true)>= 10:
            true = true.artistID.tolist()
            recall = len(set(true).intersection(pred))/len(true)
            precision = len(set(true).intersection(pred))/10
            score = score.append(pd.DataFrame(data={'userID': [i], 'recall@10': [recall], 'precision@10': [precision]}))
        else:
            fail.append(i)  
          
    return score, fail


def evaluation_report(result):
    print('{} unique user in file'.format(len(result.userID.unique())))
    score, fail = evaluation_metric(df_test, result)
    score = score[['userID','recall@10','precision@10']]
    print('{} users have less than 10 artists in test set and fail to do evaluation'.format(len(fail)))
    not_zero = score.dropna()
    not_zero = not_zero.loc[not_zero['recall@10']!= 0.0]
    print('Total number of user able to do evaluation with k=10: {}'.format(len(score)))
    print('There are {} users who has non-zero hits'.format(len(not_zero)))
    print('The average of recall@10: {}'.format(score['recall@10'].mean()))
    print('The average of precision@10: {}'.format(score['precision@10'].mean()))    

In [ ]:
#Calculate diversity
def diversity_calculation(result, merged_genre):
    diversity = pd.DataFrame(columns={'userID','diversity'})
    user_list = result.userID.unique()
    for i in user_list:
        pred = result.loc[result['userID'] == i, 'artistID'].tolist()
        genre_hit = merged_genre[merged_genre['id'].isin(pred)]
        merged_list = []
        for i in genre_hit.index:
            for item in genre_hit.genre_merge[i].strip("[]").replace("'","").replace("' ","").split(","):
                merged_list.append(item.strip())
        diversity = diversity.append(pd.DataFrame(data={'userID': [i], 'diversity': [len(set(merged_list))]}))
    return diversity   

def diversity_report(result):
    diversity = diversity_calculation(result, merged_genre)
    print('The average of diversity: {}'.format(diversity['diversity'].mean()))
    print('The median of diversity: {}'.format(diversity['diversity'].median()))                                            

In [ ]:
#Calculate novelty and coverage
def novelty_report(result, artist_sum):
    table = result.merge(artist_sum, left_on='artistID', right_on='artistID')
    print('Novelty: {}'.format(table['novelty'].mean()))

def coverage_report(result, df_train):
    coverage = len(result.artistID.unique())/len(df_train.artistID.unique())
    print('Coverage: {}'.format(coverage))

<a class="anchor" id="section61"></a>
### Section 6.1 User-based metrics

In [ ]:
user_based_result = user_based_result[user_based_result['userID'].isin(df_test.userID.unique().tolist())]

In [ ]:
evaluation_report(user_based_result)

1876 unique user in file
18 users have less than 10 artists in test set and fail to do evaluation
Total number of user able to do evaluation with k=10: 1858
There are 1494 users who has non-zero hits
The average of recall@10: 0.11444742362508752
The average of precision@10: 0.2263724434876213


In [ ]:
diversity_report(user_based_result)

The average of diversity: 20.93230277185501
The median of diversity: 22.0


In [ ]:
novelty_report(user_based_result, artist_sum)

Novelty: 0.3514609819126644


In [ ]:
coverage_report(user_based_result,df_train)

Coverage: 0.05005440696409141


<a class="anchor" id="section62"></a>
### Section 6.2 Item-based metrics

In [ ]:
item_based_result = item_based_result[item_based_result['userID'].isin(df_test.userID.unique().tolist())]

In [ ]:
evaluation_report(item_based_result)

1876 unique user in file
18 users have less than 10 artists in test set and fail to do evaluation
Total number of user able to do evaluation with k=10: 1858
There are 813 users who has non-zero hits
The average of recall@10: 0.03932209647542086
The average of precision@10: 0.07820236813778204


In [ ]:
diversity_report(item_based_result)

The average of diversity: 19.27558635394456
The median of diversity: 21.0


In [ ]:
novelty_report(item_based_result, artist_sum)

Novelty: 0.4291093274333502


In [ ]:
coverage_report(item_based_result,df_train)

Coverage: 0.09396859940929582


<a class="anchor" id="section63"></a>
### Section 6.3 Content-based 

In [ ]:
content_based_result = content_based_result[['userID','artistID']]
content_based_result = content_based_result[content_based_result['userID'].isin(df_test.userID.unique().tolist())]

In [ ]:
evaluation_report(content_based_result)

1876 unique user in file
18 users have less than 10 artists in test set and fail to do evaluation
Total number of user able to do evaluation with k=10: 1858
There are 1232 users who has non-zero hits
The average of recall@10: 0.06650510507309351
The average of precision@10: 0.1327233584499449


In [ ]:
diversity_report(content_based_result)

The average of diversity: 15.760660980810234
The median of diversity: 15.0


In [ ]:
novelty_report(content_based_result, artist_sum)

Novelty: 0.5933812509638787


In [ ]:
coverage_report(content_based_result,df_train)

Coverage: 0.1901134773822478


<a class="anchor" id="section64"></a>
### Section 6.4 PersonalRank without tags

In [ ]:
personalrank_wo_tags = personalrank_wo_tags.rename(columns={"recom_artist": "artistID"})
personalrank_wo_tags = personalrank_wo_tags[['userID','artistID']]

In [ ]:
evaluation_report(personalrank_wo_tags)

1876 unique user in file
18 users have less than 10 artists in test set and fail to do evaluation
Total number of user able to do evaluation with k=10: 1858
There are 1507 users who has non-zero hits
The average of recall@10: 0.11642438216517692
The average of precision@10: 0.22997847147470402


In [ ]:
diversity_report(personalrank_wo_tags)

The average of diversity: 21.112473347547976
The median of diversity: 22.0


In [ ]:
novelty_report(personalrank_wo_tags, artist_sum)

Novelty: 0.41507345171923604


In [ ]:
coverage_report(personalrank_wo_tags, df_train)

Coverage: 0.1268459505673869


<a class="anchor" id="section65"></a>
### Section 6.5 PersonalRank with tags

In [ ]:
personalrank_tags = personalrank_tags[['userID', "recom_artist"]]
personalrank_tags = personalrank_tags.rename(columns={"recom_artist": "artistID"})

In [ ]:
evaluation_report(personalrank_tags)

1876 unique user in file
18 users have less than 10 artists in test set and fail to do evaluation
Total number of user able to do evaluation with k=10: 1858
There are 1457 users who has non-zero hits
The average of recall@10: 0.10631854876842288
The average of precision@10: 0.21033369214208864


In [ ]:
diversity_report(personalrank_tags)

The average of diversity: 22.45309168443497
The median of diversity: 23.0


In [ ]:
novelty_report(personalrank_tags, artist_sum)

Novelty: 0.3413813787965046


In [ ]:
coverage_report(personalrank_tags, df_train)

Coverage: 0.07088450178765739
